In [ ]:
from get_audio import AudioFetcher

url = # 'https://youtu.be/HtSuA80QTyo'
audio_fetcher = AudioFetcher(url)
save_location = 'data/fetchedAudio.webm'
audio_fetcher.fetch(save_location)
wav_location = 'data/ConvertedToWav.wav'
audio_fetcher.convert_to_wav(wav_location)

In [ ]:
# UPLOADING THE CAPTIONS DATA DIRECTLY CAUSE THIS AINT WORKING FOR SOME REASON.

from get_cc import CaptionFetcher

# url = # https://www.youtube.com/api/timedtext?v=HtSuA80QTyo&asr_langs=de%2Cen%2Ces%2Cfr%2Cid%2Cit%2Cja%2Cko%2Cnl%2Cpt%2Cru%2Ctr%2Cvi&caps=asr&exp=xftt%2Cxctw&xoaf=5&hl=en&ip=0.0.0.0&ipbits=0&expire=1631187357&sparams=ip%2Cipbits%2Cexpire%2Cv%2Casr_langs%2Ccaps%2Cexp%2Cxoaf&signature=8D60161D31A8301B3B8524EFD3F22E5D4E66957A.2BF728B067CB1EA262D8FCF9E898F109FBE0A69B&key=yt8&lang=en&fmt=json3&xorb=2&xobt=3&xovt=3

# save_location = 'data/captions_data.json'
caption_fetcher = CaptionFetcher(url)
# caption_fetcher.fetch(save_location)

In [ ]:
# Create a new json

from create_jsons import create_jsons
create_jsons('data/captions_data.json', 'data/dataset.json')

In [ ]:
# Chopping clips

from chop_audio import AudioChopper

chopper = AudioChopper(wav_location=wav_location, save_location='data/clips/', time_stamps_location='data/captions_data.json')

In [ ]:
# Clean The Data

caption_fetcher.clean_data('data/dataset.json')

# MODEL